## Lyrics data

In [ ]:
from vagalume import lyrics
import pandas as pd
import difflib as di
import time

In [ ]:
data_GI_AF_AA_FO = pd.read_csv('preprocessing_data.csv',index_col=0)

### Lyrics search

In [ ]:
artistas = data_GI_AF_AA_FO[['artist']].values 
musica = data_GI_AF_AA_FO[['track']].values

letra = []
list_error = []
similarity = []
k=0
j=0
for i in range(0,len(artistas)): 
    time.sleep(2)
    if i%50 ==0:
        print("Waiting a little")
        print("Lyrics searched: ", i)
        print("Downloaded lyrics: ", k)
        time.sleep(5)
        
    artist_name = artistas[i][0]
    song_name = musica[i][0]
    print(artist_name,song_name)
    
    try:
        result = lyrics.find(artist_name, song_name)
        if result.is_not_found():
            letra.append(None)
            similarity.append(None)
        else:
            s_artist = di.SequenceMatcher(None, artist_name.upper(), result.artist.name.upper()).ratio()
            s_name = di.SequenceMatcher(None, song_name.upper(), result.song.name.upper()).ratio()
            similarity.append({'name_similarity':s_name, 'artist_similarity':s_artist})
            print(s_name,song_name.upper(), result.song.name.upper())
            letra.append([result.song.name, result.artist.name,result.song.lyric])
            k+=1
            
    except:
        letra.append(None)
        similarity.append(None)
        time.sleep(5)
#         

In [ ]:
data_GI_AF_AA_FO_L = data_GI_AF_AA_FO.copy()
if len(letra)== len(artistas):
    data_GI_AF_AA_FO_L['lyrics'] = letra
    data_GI_AF_AA_FO_L['lyrics_confidence'] = similarity

### Similarity filter

In [ ]:
count =data_GI_AF_AA_FO_L['lyrics'].value_counts()
err=0
exclude=0
for i in range(0,len(letra)):
    if data_GI_AF_AA_FO_L['lyrics_confidence'][i] != None:
        if data_GI_AF_AA_FO_L['lyrics_confidence'][i]['name_similarity']<0.9:
             
            track_name = data_GI_AF_AA_FO_L['track'][i].upper()

            track_name = track_name.replace("AO VIVO", "")
            
            new_similarity = di.SequenceMatcher(None, track_name, data_GI_AF_AA_FO_L['lyrics'][i][0].upper()).ratio()
           
            if new_similarity <0.9:
                print('Spotify','      |      ','Vagalume')
                print()
                print(data_GI_AF_AA_FO_L['artist'][i],'  |  ', data_GI_AF_AA_FO_L['lyrics'][i][1])
                print(data_GI_AF_AA_FO_L['track'][i],'  |  ', data_GI_AF_AA_FO_L['lyrics'][i][0])
                print(similarity[i])
                print()
                print("Do you want to get rid of this lyric?")
                apagar = input()
                if apagar == "yes":
                    data_GI_AF_AA_FO_L['lyrics'][i]=None
                    data_GI_AF_AA_FO_L['lyrics_confidence'][i]=None
                    print('The lyrics of this track were successfully deleted.')
                    exclude+=1                    
                err+=1
                print()
                print()

In [ ]:
data_GI_AF_AA_FO_L.to_csv('lyrics_preprocessing_spotify_data.csv')